<a href="https://colab.research.google.com/github/buganart/mocogan/blob/master/mocogan_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`

In [ ]:
#@markdown Check GPU, should be a Tesla V100
!nvidia-smi -L
import os
print(f"We have {os.cpu_count()} CPU cores.")

In [ ]:
#@markdown Mount google drive
from google.colab import drive
from google.colab import output
drive.mount('/content/drive')

from pathlib import Path
if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
    raise RuntimeError(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Go to the google drive web UI\n"
        "\t2. Right click shared folder IRCMS_GAN_collaborative_database and click \"Add shortcut to Drive\""
    )

def clear_on_success(msg="Ok!"):
    if _exit_code == 0:
        output.clear()
        print(msg)

In [ ]:
#@markdown Install wandb and log in
%pip install wandb
output.clear()
import wandb
from pathlib import Path
wandb_drive_netrc_path = Path("drive/My Drive/colab/.netrc")
wandb_local_netrc_path = Path("/root/.netrc")
if wandb_drive_netrc_path.exists():
    import shutil

    print("Wandb .netrc file found, will use that to log in.")
    shutil.copy(wandb_drive_netrc_path, wandb_local_netrc_path)
else:
    print(
        f"Wandb config not found at {wandb_drive_netrc_path}.\n"
        f"Using manual login.\n\n"
        f"To use auto login in the future, finish the manual login first and then run:\n\n"
        f"\t!mkdir -p '{wandb_drive_netrc_path.parent}'\n"
        f"\t!cp {wandb_local_netrc_path} '{wandb_drive_netrc_path}'\n\n"
        f"Then that file will be used to login next time.\n"
    )

!wandb login
output.clear()
print("ok!")

# Description


In [ ]:
#@title Configuration

#@markdown Directories can be found via file explorer on the left by navigating into `drive` to the desired folders. 
#@markdown Then right-click and `Copy path`.
#@markdown - The video database directory
video_db_dir = "/content/drive/My Drive/VIDEOS/SINGLE_FILE/VIDEO1" #@param {type:"string"}
# video_db_dir = "/content/drive/My Drive/VIDEOS/RESIZED/SINGLE_FILE/VIDEO1" #@param {type:"string"}
#@markdown - Optional: The folder to save resized(preprocessed) videos
#@markdown  - If resize_save_dir="", videos will not be saved and will be lost when runtime timeout.
#@markdown  - If resize_save_dir==video_db_dir, assume the videos are already resized (no preprocessing).
resize_save_dir = "/content/drive/My Drive/VIDEOS/RESIZED/SINGLE_FILE/VIDEO1" #@param {type:"string"}
# experiment_dir = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-violingan/melgan" #@param {type:"string"}
#@markdown - The output directory of generated videos 
out_dir = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-mocogan" #@param {type:"string"}
#@markdown - set wandb run id of logged run to resume from there
resume_id = "" #@param {type:"string"}
#@markdown - set batch_size, default: 16
batch_size = 16 #@param {type:"integer"}
#@markdown - set num of iterations, default: 120000
n_iter = 120000 #@param {type:"integer"}
#@markdown - learning rate of the model (all components)
lr = 0.0002 #@param {type:"number"}
#@markdown - the height and width of input (resized) video
img_size = 96 #@param {type:"integer"}
#@markdown - the number of channel of input (resized) video
nc = 3 #@param {type:"integer"}

#@markdown ##model parameters
ndf = 64 #@param {type:"integer"}
ngf = 64 #@param {type:"integer"}
d_E = 10 #@param {type:"integer"}
hidden_size = 100 #@param {type:"integer"}
d_C = 50 #@param {type:"integer"}
d_M = 10 #@param {type:"integer"}

#@markdown ##log parameters
#@markdown - the number of epoch to save 1 generated video (default:1000)
video_epoch = 1000 #@param {type:"integer"}
#@markdown - the number of epoch to save model checkpoint (default:10000)
checkpoint_epoch = 5000 #@param {type:"integer"}

In [ ]:
#@title Clone mocogan repo

!git clone https://github.com/buganart/mocogan

In [ ]:
#@title Install Dependencies

%cd /content/mocogan
# %pip install -r requirements.txt
%pip install sk-video
%pip install wandb
clear_on_success("Dependencies installed.")

In [ ]:
#@title Copy video files to runtime
if resize_save_dir == video_db_dir:
    local_wav_dir = Path("/content/mocogan/resized_data")
else:
    local_wav_dir = Path("/content/mocogan/raw_data")
!find "{video_db_dir}" -maxdepth 1 -type f | xargs -t -d "\n" -I'%%' -P 10 -n 1 rsync -a '%%' "$local_wav_dir"/
clear_on_success("All files copied to this runtime.")


In [ ]:
#@title RESIZE

import glob
import os

if resize_save_dir != video_db_dir:
    current_path = "/content/mocogan/"
    resized_path = os.path.join(current_path, 'resized_data')
    files = glob.glob(os.path.join(current_path, 'raw_data/*.*'))

    #remove all files in resized_path first
    for x, file_path in enumerate(glob.glob(os.path.join(resized_path, '*.*'))):
        try:
            os.remove(file_path)
        except OSError as e:
            print("Error: %s : %s" % (file_path, e.strerror))

    for i, file in enumerate(files):
        resized_filename = os.path.join(resized_path, str(i))
        !ffmpeg -i "{file}" -pix_fmt yuv420p -vf scale="{img_size}":"{img_size}" "{resized_filename}".mp4

    # crop=96:96:42:24

    if resize_save_dir:
        resize_save_dir = Path(resize_save_dir)
        resize_save_dir.mkdir(parents=True, exist_ok=True)
        # copy files to processed dir for resize_save_dir
        !find "{resized_path}" -maxdepth 1 -type f | xargs -t -d "\n" -I'%%' -P 10 -n 1 rsync -a '%%' "$resize_save_dir"/

    clear_on_success("All files resized.")

In [ ]:
#@title TRAIN
# os.environ["WANDB_MODE"] = "dryrun"

!python train.py \
--out_dir "$out_dir" \
--resume_id "$resume_id" \
--batch_size "$batch_size" \
--niter "$n_iter" \
--lr "$lr" \
--img_size "$img_size" \
--nc "$nc" \
--ndf "$ndf" \
--ngf "$ngf" \
--d_E "$d_E" \
--hidden_size "$hidden_size" \
--d_C "$d_C" \
--d_M "$d_M" \
--video_epoch "$video_epoch" \
--checkpoint_epoch "$checkpoint_epoch"